<a href="https://colab.research.google.com/github/fbertran/m2csmi/blob/master/Digue.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
from openturns import *
import openturns as ot
import random
import math
from math import sqrt

#When the river height exceeds that of the dyke, flooding occurs.

####################
# Flooding function
####################

# Create here the python lines to define the implementation of the function

# In order to be able to use that function with the openturns library,
# it is necessary to define a class which derives from OpenTURNSPythonFunction

class modelePYTHON(OpenTURNSPythonFunction) :
# that following method defines the input size (4) and the output size (1)
	def __init__(self) :
		OpenTURNSPythonFunction.__init__(self,8,1)

# that following method gives the implementation of modelePYTHON
	def _exec(self,x) :
		Q=x[0]
		if Q<500.:
			Q=500.
		elif Q>3000.:
			Q=3000.
		Ks=x[1]
		if Ks<15.:
			Ks=15.
		Zv=x[2]
		Zm=x[3]
		Hd=x[4]
		Cb=x[5]
		L=x[6]
		B=x[7]
		H = (Q/(B*Ks*sqrt((Zm-Zv)/L)))**0.6
		S = Zv + H - Hd -Cb
		return [S]

# Use that function defined in the script python
# with the openturns library
# Create a NumericalMathFunction from modelePYTHON
flooding = NumericalMathFunction(modelePYTHON())


























In [0]:
####################
# Cost function
####################

class modelePYTHON2(OpenTURNSPythonFunction) :
# that following method defines the input size (8) and the output size (1)
	def __init__(self) :
		OpenTURNSPythonFunction.__init__(self,8,1)

# that following method gives the implementation of modelePYTHON
	def _exec(self,x) :
		Q=x[0]
		Ks=x[1]
		Zv=x[2]
		Zm=x[3]
		Hd=x[4]
		Cb=x[5]
		L=x[6]
		B=x[7]
		S = flooding(x)[0];
		Cp = 0.
		if S>0.:
			Cp += 1.
		else:
			Cp += 0.2 + 0.8*(1-math.exp(-1000/S**4))
		if Hd>8.:
			Cp += Hd/20.
		else:
			Cp += 8./20.
		return [Cp]

# Use that function defined in the script python
# with the openturns library
# Create a NumericalMathFunction from modelePYTHON
cost = NumericalMathFunction(modelePYTHON2())


In [0]:
#######################
### Input random vector
#######################

#distributionQ = Gumbel(1013.,558.)
distributionQ = Gumbel(1./558.,1013.)
distributionKs = Normal(30.,8.)
distributionZv = Triangular(49.,50.,51.)
distributionZm = Triangular(54.,55.,56.)
distributionHd = Uniform(7.,9.)
distributionCb = Triangular(55.,55.5,56.)
distributionL = Triangular(4990.,5000.,5010.)
distributionB = Triangular(295.,300.,305.)

pdfLoiQ = distributionQ.drawPDF()
pdfLoiQ.draw("distributionQ_pdf", 640, 480)


collectionMarginals = [distributionQ,distributionKs,distributionZv,distributionZm,distributionHd,distributionCb,distributionL,distributionB]
aCopula = IndependentCopula(len(collectionMarginals))
aCopula.setName("Independent copula")

inputDistribution = ComposedDistribution(collectionMarginals, Copula(aCopula))
inputDistribution.setDescription(("Q","Ks","Zv","Zm","Hd","Cb","L","B"))

inputRandomVector = RandomVector(inputDistribution)
inputRandomVector.setDescription(("Q","Ks","Zv","Zm","Hd","Cb","L","B"))

outputS = RandomVector(flooding, inputRandomVector)
outputCp = RandomVector(cost, inputRandomVector)

In [0]:
###############################
# a) Probabilite de flooding
##############################

RandomGenerator.SetSeed(random.randint(1,100))

myEvent = Event(outputS, Greater(), -6.0)
myEvent.setName("Flooding!")

print("################")
print("Monte Carlo")
print("################")
print("")

maximumOuterSampling = 100000
blockSize = 1
coefficientOfVariation = 0.1

# Creation de l'algorithme Monte Carlo
algoMC = MonteCarlo(myEvent)
algoMC.setMaximumOuterSampling(maximumOuterSampling)
algoMC.setBlockSize(blockSize)
algoMC.setMaximumCoefficientOfVariation(coefficientOfVariation)

algoMC.setConvergenceStrategy(Full())
# For statistics about the algorithm
initialNumberOfCall = flooding.getEvaluationCallsNumber()


algoMC.run()

result = algoMC.getResult()
print("MC estimate of the probability of flooding : ", result.getProbabilityEstimate())
print("Variance of the MC estimator : ", result.getVarianceEstimate())
print('CV =', result.getCoefficientOfVariation())
#print('MonteCarlo result=', result)
print('Number of executed iterations =', result.getOuterSampling())
print('Number of calls to the flooding function =', flooding.getEvaluationCallsNumber() - initialNumberOfCall)


In [0]:
################################
# b) Sobol Indices
################################

size = 10000

###For the Flooding
inputDesignS = ot.SobolIndicesAlgorithmImplementation.Generate(inputDistribution, size, True)
outputDesignS = flooding(inputDesignS)
sensitivityAnalysisS = ot.SaltelliSensitivityAlgorithm(inputDesignS, outputDesignS, size)

#First order Sobol indices
print("For the flooding:")
print("First Order Sobol Indices: ", sensitivityAnalysisS.getFirstOrderIndices())
#Total order Sobol indices
print("Total Order Sobol Indices: ", sensitivityAnalysisS.getTotalOrderIndices())

#Graph file
graph = sensitivityAnalysisS.draw()
graph.draw("indices_sobol_S.png")


###For the cost
size = 30000

inputDesignCp = ot.SobolIndicesAlgorithmImplementation.Generate(inputDistribution, size, True)
outputDesignCp = cost(inputDesignCp)
sensitivityAnalysisCp = ot.SaltelliSensitivityAlgorithm(inputDesignCp, outputDesignCp, size)

#First order Sobol indices
print("For the cost:")
print("First Order Sobol Indices:", sensitivityAnalysisCp.getFirstOrderIndices())
#Total order Sobol indices
print("Total Order Sobol Indices: ", sensitivityAnalysisCp.getTotalOrderIndices())

#Graph file
graph = sensitivityAnalysisCp.draw()
graph.draw("indices_sobol_Cp.png")

In [0]:
##########################
# c) Metamodel
##########################

#A LatinHyperSquare should be used instead of random points.
X = inputRandomVector.getSample(100)
dim = X.getDimension()

Y1 = flooding(X)
Y2 = cost(X)

#Just one of the possible covariance model
covarianceModel = SquaredExponential(dim)

basis = ConstantBasisFactory(dim).build()

algoS = KrigingAlgorithm(X, Y1, covarianceModel, basis)
algoCp = KrigingAlgorithm(X, Y2, covarianceModel, basis)

algoS.run()
algoCp.run()
resultS = algoS.getResult()
resultCp = algoCp.getResult()
metamodelS = resultS.getMetaModel()
metamodelCp = resultCp.getMetaModel()

In [0]:
Xtest = inputRandomVector.getSample(10)
print("For Flooding with 10 random test points")
print("With metamodel:")
print(metamodelS(Xtest))
print("True value: ")
print(flooding(Xtest))

In [0]:
Xtest = inputRandomVector.getSample(10)
print("For Cost with 10 random test points")
print("With metamodel:")
print(metamodelCp(Xtest))
print("True value: ")
print(cost(Xtest))